In [4]:
import torch
feature = torch.rand(64, 16, 21, 31)
print(feature.shape, feature)

torch.Size([64, 16, 21, 31]) tensor([[[[0.8164, 0.2177, 0.0429,  ..., 0.9776, 0.1016, 0.2746],
          [0.4883, 0.6037, 0.7285,  ..., 0.2606, 0.9478, 0.8684],
          [0.3729, 0.9277, 0.0939,  ..., 0.1225, 0.1570, 0.2840],
          ...,
          [0.2387, 0.5544, 0.8949,  ..., 0.7868, 0.8589, 0.7702],
          [0.2329, 0.7147, 0.0774,  ..., 0.4690, 0.5391, 0.9492],
          [0.0057, 0.8380, 0.3247,  ..., 0.2195, 0.2565, 0.6381]],

         [[0.9483, 0.1532, 0.4932,  ..., 0.2996, 0.7947, 0.5775],
          [0.2674, 0.2898, 0.5669,  ..., 0.1886, 0.7102, 0.8192],
          [0.8406, 0.5492, 0.3811,  ..., 0.0671, 0.1839, 0.8072],
          ...,
          [0.6060, 0.5777, 0.7684,  ..., 0.0608, 0.0051, 0.4133],
          [0.6707, 0.2610, 0.0911,  ..., 0.8278, 0.7188, 0.9204],
          [0.8398, 0.5947, 0.7888,  ..., 0.2318, 0.3437, 0.5497]],

         [[0.7684, 0.3527, 0.4703,  ..., 0.2823, 0.7186, 0.7168],
          [0.2384, 0.2763, 0.4731,  ..., 0.0476, 0.8556, 0.9075],
          [0.

In [5]:
flattened = feature.view(-1, 31 * 21)
print(flattened.shape, flattened)

torch.Size([1024, 651]) tensor([[0.8164, 0.2177, 0.0429,  ..., 0.2195, 0.2565, 0.6381],
        [0.9483, 0.1532, 0.4932,  ..., 0.2318, 0.3437, 0.5497],
        [0.7684, 0.3527, 0.4703,  ..., 0.2182, 0.0777, 0.6874],
        ...,
        [0.1528, 0.6144, 0.7693,  ..., 0.3855, 0.5034, 0.6154],
        [0.8887, 0.2922, 0.9445,  ..., 0.2270, 0.5280, 0.9234],
        [0.5701, 0.4048, 0.0060,  ..., 0.7747, 0.9871, 0.7717]])


In [8]:
import torch.nn.functional as F
softmax_attention = F.softmax(flattened/1.0, dim=-1)
print(softmax_attention.shape, softmax_attention)

torch.Size([1024, 651]) tensor([[0.0020, 0.0011, 0.0009,  ..., 0.0011, 0.0011, 0.0017],
        [0.0023, 0.0010, 0.0015,  ..., 0.0011, 0.0013, 0.0016],
        [0.0019, 0.0013, 0.0014,  ..., 0.0011, 0.0010, 0.0018],
        ...,
        [0.0010, 0.0016, 0.0019,  ..., 0.0013, 0.0014, 0.0016],
        [0.0022, 0.0012, 0.0023,  ..., 0.0011, 0.0015, 0.0023],
        [0.0016, 0.0013, 0.0009,  ..., 0.0019, 0.0024, 0.0019]])


In [10]:
import numpy as np
pos_x, pos_y = np.meshgrid(
        np.linspace(-1., 1., 21),
        np.linspace(-1., 1., 31)
        )
pos_x = torch.from_numpy(pos_x.reshape(31*21)).float()
pos_y = torch.from_numpy(pos_y.reshape(31*21)).float()
expected_x = torch.sum(pos_x*softmax_attention, dim=1, keepdim=True)
expected_y = torch.sum(pos_y*softmax_attention, dim=1, keepdim=True)

print(expected_x.shape, expected_y.shape)

torch.Size([1024, 1]) torch.Size([1024, 1])


In [14]:
depth = torch.rand(64,1,48,64)
flattened_depth = depth.view(-1, 64 * 48)
print(flattened_depth.shape)

torch.Size([64, 3072])


In [17]:
coord = expected_x * 32 + 31 + (expected_y * 24 + 23) * 64
print(coord.shape, coord)

torch.Size([1024, 1]) tensor([[1490.5820],
        [1508.5626],
        [1508.1868],
        ...,
        [1502.4368],
        [1488.0676],
        [1499.1844]])


In [18]:
rc = torch.reshape(coord, (64, 16))
rc = torch.round(rc).long()
print(rc.shape, rc)

torch.Size([64, 16]) tensor([[1491, 1509, 1508,  ..., 1478, 1511, 1496],
        [1513, 1497, 1513,  ..., 1519, 1508, 1488],
        [1505, 1502, 1505,  ..., 1511, 1496, 1518],
        ...,
        [1494, 1499, 1503,  ..., 1518, 1496, 1524],
        [1507, 1488, 1497,  ..., 1512, 1510, 1516],
        [1503, 1508, 1508,  ..., 1502, 1488, 1499]])


In [22]:
z = torch.take(flattened_depth, rc)
print(z.shape, z)

torch.Size([64, 16]) tensor([[0.3682, 0.2554, 0.5870,  ..., 0.1919, 0.3736, 0.9061],
        [0.3642, 0.9807, 0.3642,  ..., 0.0481, 0.5870, 0.2937],
        [0.5287, 0.9172, 0.5287,  ..., 0.3736, 0.9061, 0.9376],
        ...,
        [0.0202, 0.5699, 0.9238,  ..., 0.9376, 0.9061, 0.2541],
        [0.8043, 0.2937, 0.9807,  ..., 0.2765, 0.5827, 0.3894],
        [0.9238, 0.5870, 0.5870,  ..., 0.9172, 0.2937, 0.5699]])


In [25]:
flattened_depth[0, 1478]

tensor(0.1919)

In [26]:
expected_x

tensor([[ 0.0038],
        [ 0.0033],
        [-0.0053],
        ...,
        [ 0.0034],
        [-0.0050],
        [-0.0101]])

In [37]:
r_expected_x = torch.reshape(expected_x, (1, 64 * 16))
r_expected_y = torch.reshape(expected_y, (1, 64 * 16))
Z = torch.reshape(z, (1, 64 * 16))
print(r_expected_x.shape, r_expected_x)

torch.Size([1, 1024]) tensor([[ 0.0038,  0.0033, -0.0053,  ...,  0.0034, -0.0050, -0.0101]])


In [38]:
f = 450.0
X = Z / f * r_expected_x
Y = Z / f * r_expected_y
print(X.shape, Y.shape)

torch.Size([1, 1024]) torch.Size([1, 1024])


In [40]:
output = torch.cat((X,Y,Z), 0)
print(output.shape)

torch.Size([3, 1024])


In [66]:
print(output.shape, output)

torch.Size([3, 1024]) tensor([[ 3.0790e-06,  1.8891e-06, -6.9771e-06,  ...,  6.9582e-06,
         -3.2779e-06, -1.2797e-05],
        [-6.6799e-06,  2.0160e-06,  4.5503e-06,  ..., -8.9243e-07,
         -6.2762e-06, -2.8791e-06],
        [ 3.6824e-01,  2.5540e-01,  5.8702e-01,  ...,  9.1723e-01,
          2.9368e-01,  5.6985e-01]])


In [68]:
ct = torch.transpose(output, 1, 0)
print(ct.shape)
# ct = torch.reshape(ct, (48, 64))
# ct = ct.permute(2, 0, 1)
print(ct)

torch.Size([1024, 3])
tensor([[ 3.0790e-06, -6.6799e-06,  3.6824e-01],
        [ 1.8891e-06,  2.0160e-06,  2.5540e-01],
        [-6.9771e-06,  4.5503e-06,  5.8702e-01],
        ...,
        [ 6.9582e-06, -8.9243e-07,  9.1723e-01],
        [-3.2779e-06, -6.2762e-06,  2.9368e-01],
        [-1.2797e-05, -2.8791e-06,  5.6985e-01]])


In [70]:
ct = torch.transpose(torch.reshape(torch.reshape(ct, (1, 3072)), (48, 64)), 1, 0)
print(ct.shape, ct)

torch.Size([64, 48]) tensor([[ 3.0790e-06,  7.3684e-06,  7.0882e-01,  ..., -7.4541e-06,
          6.1931e-06,  2.0213e-02],
        [-6.6799e-06,  2.7796e-01, -1.9122e-06,  ...,  1.4597e-06,
          3.5801e-01, -2.3746e-05],
        [ 3.6824e-01, -1.9344e-05,  1.0800e-06,  ...,  7.0882e-01,
         -6.1404e-06, -2.7265e-05],
        ...,
        [-8.7943e-06,  5.8702e-01, -4.1606e-06,  ...,  1.5940e-05,
          2.0213e-02, -1.2797e-05],
        [ 9.0611e-01,  1.1934e-06, -2.2334e-06,  ...,  8.2196e-01,
         -3.2235e-07, -2.8791e-06],
        [-3.8233e-07,  5.1653e-07,  3.2809e-01,  ..., -6.8265e-07,
         -2.6091e-07,  5.6985e-01]])


In [64]:
output[:,9]

tensor([-9.2672e-06, -1.1165e-05,  4.6974e-01])

In [65]:
ct[29:32, 0]

tensor([ 3.2809e-01, -2.2862e-05, -2.8824e-05])

In [62]:
ct[:, 9, 0]

tensor([-6.4296e-06, -3.5192e-06,  5.6985e-01])

In [54]:
ct[:,0, 0]

tensor([ 3.0790e-06, -6.6799e-06,  3.6824e-01])

In [55]:
x = torch.rand(1,1024)
x

tensor([[0.2493, 0.7042, 0.1844,  ..., 0.5085, 0.2837, 0.9105]])

In [58]:
x = torch.rand(1,1024)
x = torch.reshape(x, (64, 16))
x = torch.transpose(x, 1, 0)
y = torch.rand(1,1024)
y = torch.reshape(y, (64, 16))
y = torch.transpose(y, 1, 0)
z = torch.rand(1,1024)
z = torch.reshape(z, (64, 16))
z = torch.transpose(z, 1, 0)

tensor([[0.2493, 0.7042, 0.1844,  ..., 0.7844, 0.7588, 0.6657],
        [0.8313, 0.1836, 0.2371,  ..., 0.5157, 0.1571, 0.3898],
        [0.3626, 0.2576, 0.6578,  ..., 0.3195, 0.2390, 0.6638],
        ...,
        [0.6438, 0.8383, 0.5530,  ..., 0.3265, 0.3717, 0.3140],
        [0.5593, 0.4217, 0.8428,  ..., 0.6328, 0.2114, 0.0917],
        [0.2612, 0.9464, 0.5104,  ..., 0.5085, 0.2837, 0.9105]])